<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>François Rechenmann &amp; Thierry Parmentelat</span>
<span><img src="media/inria-25.png" /></span>
</div>

# Algorithme UPGMA

Nous allons voir dans ce complément une implémentation en python de l'algorithme UPGMA qui a vient d'être expliqué dans la vidéo. Mais comme toujours commençons par ceci. 

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Distance de Needleman et Wunsch

Nous avons bien sûr besoin de la fonction `distance` de Needleman et Wunsch, que nous importons du notebook de la semaine passée&nbsp;:

In [ ]:
from w4_s09_c1_needleman_wunsh_iter import distance

### Format du fichier d'entrée

Nous allons cette fois utiliser un fichier d'entrée dans un format légèrement différent, de façon à pouvoir associer de façon simple un nom aux différents fragments d'ADN que nous manipulons. Pour cela, le fichier d'entrée va ressembler à ceci (ce sont bien sûr des données fantaisistes)&nbsp;: 

In [ ]:
with open("data/named-species.txt") as input:
    for line in input:
        print(line, end="")

C'est à dire que chaque ligne contient maintenant, séparé par un ou plusieurs espaces, un nom et une séquence d'ADN. Pour cette raison nous ne pouvons pas réutiliser telle quelle la fonction qui élabore le tableau des distances, nous allons récrire cette parties.

### Quelques outils python

##### La méthode `split`

Nous allons utiliser la méthode `split` sur les chaines en python, qui découpe simplement une chaine en morceaux&nbsp;:

In [ ]:
chaine = "BABAAB  ADAD"
chaine.split()

Comme on le voit, sans argument cette méthode fait exactement ce dont nous allons avoir besoin pour découper une ligne d'entrée en deux partie. 

##### La méthode `remove` pour enlever d'une liste

Nous utilisons également la méthode `remove` sur les listes, qui fonctionne un peu comme `append`, mais possède l'effet inverse d'enlever un élément d'une liste&nbsp;:

In [ ]:
# une liste avec des chaines et des tuples
l = [ 'a', 'b', (1, 2)]
print("début:", l)
# avec remove on peut enlever une chaine
l.remove('a')
print("milieu:", l)
# ou on peut enlever un tuple
l.remove( (1, 2))
print("fin:", l)

##### Tester si une clé est présente dans un dictionnaire

Pour tester si un dictionnaire possède ou non une clé, on utilise l'opérateur `in`&nbsp;:

In [ ]:
d = {'a' : 1, (1, 2) : 'un-deux' }
(1, 2) in d

In [ ]:
'a' in d

In [ ]:
'b' in d

### Structures de données

Pour écrire l'algorithme nous allons utiliser les structures de données suivantes. 

##### Espèces

Pour représenter les différentes espèces qui entrent en jeu, nous allons utiliser&nbsp;:

  * soit directement une chaine, pour les espèces qui sont présentes dans le fichier d'entrée, dites **espèces natives**,
  * soit un tuple de deux espèces pour les espèces qui sont créés par l'algorithme au fur et à mesure de sa progression, qu'on appellera des **espèces synthétiques**.

Ainsi par exemple une espèce pourra être 

 * soit `spam`,
 * soit `('eggs', 'bread')` pour représenter l'ancêtre commun aux espèces `eggs` et `bread`,
 * soit encore un étage au-dessus `(('eggs', 'bread'), ('bacon', 'chicken'))`

##### `native_species`

Cette variable est simplement le résultat de l'analyse du fichier d'entrée, c'est simplement un dictionnaire `name` $\rightarrow$ `dna`.

##### `distances`

Exactement comme on l'a vu dans le complément précédent sur le tableau des distances, cette variable va mémoriser les distances entre deux espèces, natives ou synthétiques. Comme dans ce cas-là, ceci sera implémenté comme un dictionnaire indexé sur un tuple d'espèces, et on évite la duplication au moyen d'une fonction de commodité `get_distance`. Évidemment on l'initialise avec uniquement les espèces natives, puis on ajoute au fur et à mesure toutes les distances nécessaires.

##### `species`

Cette variable jour un rôle clé, c'est la liste des espèces qui sont encore *en lice*. On l'initialise avec les espèces natives, et à chaque création d'une espèce synthétique on:

* **ajoute la nouvelle espèce**, et on
* **enlève les deux espèces filles** (natives ou synthétiques) qui la composent.

C'est donc cette variable qui détermine la fin de l'algorithme (lorsqu'il n'y a plus qu'une espèce), et qui à ce stade contient le résultat final.

### Commodités pour accéder aux distances

##### Retrouver une distance dans la table

Notre structure `distances` utilise la symétrie de la fonction de distances, et ne contient que la moitié du tableau. 
Contrairement à ce qu'on avait pu faire dans la première version du tableau de distances (séquence 3 de cette même semaine), on ne dispose plus d'un ordre total sur les espèces (souvenez-vous, nous avions cette fois-là représenté les espèces par un indice).

Pour redire la même chose autrement, pour chercher la distance entre `sp1` et `sp2`, on ne peut pas savoir *a priori* s'il faut chercher dans le dictionnaire en utilisant la clé `(sp1, sp2)` ou la clé `(sp2, sp1)`. Mais ce n'est pas très grave, il nous suffit d'essayer les deux options.

On peut ainsi récrire la fonction de recherche de distance dans la table&nbsp;:

In [ ]:
def get_distance(distances, sp1, sp2):
    """
    Cherche la distance entre les espèces sp1 et sp2 
    dans la table distances
    """
    if (sp1, sp2) in distances:
        return distances[ (sp1, sp2) ]
    # sinon il doit y être dans l'autre sens
    else:
        return distances[ (sp2, sp1) ]
    # En principe, si notre algorithme est correct on n'a pas 
    # besoin de considérer d'autre cas comme sp1 == sp2 

##### Calcul de la distance minimale

Nous allons également avoir besoin d'une fonction utilitaire qui recherche la distance minimale dans tous les couples d'espèces encore en course. Pour cela nous avons besoin

 * bien entendu de `distances`, 
 * et de la liste `species` des espèces à prendre en compte, ceci parce qu'il est plus simple de tout conserver dans `distances`, même lorsque des espèces sont dissoutes dans un espèce synthétique plus grosse. De ce fait, la liste `species` est strictement plus petite que la liste des espèces qui apparaissent dans `distance`.
 
Voici une implémentation possible pour la fonction `minimal_couple`, qui recherche parmi tous les couples d'espèces dans `species` celui qui correspond à la distance minimale.

In [ ]:
def minimal_couple(distances, species):
    """
    Recherche parmi tous les couples de species x species
    avec sp1 != sp2 
    celui qui correspond à la distance minimale dans distances
    
    Retourne le couple en question - et pas la distance 
    car on n'en a pas besoin
    """
    ### initialisations
    # le couple résultat
    couple = None
    # la valeur minimale rencontrée jusqu'ici
    minimum = None
    # on balaie tous les couples
    for sp1 in species:
        for sp2 in species:
            # on ne considère que les couples qui sont dans `distances`
            # de cette façon
            # (*) on évite le cas où sp1 == sp2, et
            # (*) on ne traite qu'une fois chaque paire
            if (sp1, sp2) not in distances:
                continue
            # si minimum est None, on traite notre premier couple
            if minimum is None:
                minimum = get_distance(distances, sp1, sp2)
                couple = sp1, sp2
            # sinon, on mémorise ce couple si sa distance est 
            # inférieure au minimum courant
            else:
                candidate = get_distance(distances, sp1, sp2)
                if candidate < minimum:
                    minimum = candidate
                    couple = sp1, sp2
    # on n'oublie pas de retourner le résultat
    return couple

### L'algorithme UPGMA

Il ne nous reste plus qu'à mettre tout ensemble. Par commodité on prévoit un paramètre d'entrée supplémentaire `verbose` qui, s'il est vrai, donne une sortie plus bavarde&nbsp;:

In [ ]:
def UPGMA(filename, verbose=False):
    """
    Lit un fichier contenant sur chaque ligne 
    un nom d'espèce et un ADN
    
    Le paramètre optionnel verbose permet de déclencher 
    des impressions qui illustrent la progression de l'algorithme

    Calcule le tableau des distances, 
    puis implémente l'algorithme UPGMA
    
    Renvoie l'arbre de filiation sous forme d'un tuple 
    de tuples / et noms d'espèces
    """
    
    native_species = {}

    # lire le fichier
    with open(filename) as input:
        for line in input:
            # on coupe la ligne pour trouver le nom et la séquence
            name, dna = line.split()
            # on mémorise dans native_species
            native_species[name] = dna
    
    # on calcule le tableau des distances
    distances = {}

    # essentiellement comme on l'a fait dans la séquence passée
    for sp1, dna1 in native_species.items():
        for sp2, dna2 in native_species.items():
            # on ignore les couples diagonaux
            if sp1 == sp2:
                continue
            # la seule astuce est de regarder si le couple
            # symétrique est déjà présent dans distances
            # auquel cas on ignore aussi
            if (sp2, sp1) in distances:
                continue
            distances[sp1, sp2] = distance(dna1, dna2)
    
    # la liste des clés de départ
    # convertie en list pour python3
    # de façon à pouvoir ôter les espèces de cette liste
    # au fur et à mesure du parcours
    species = list(native_species.keys())
    
    # bavardage
    if verbose:
        print(10*'+', 'Initial distances')
        print(distances )

    # c'est ici que se passe la partie intéressante
    # on s'attend à ce que species se réduise jusqu'à ne contenir plus que 
    # une unique espèce synthétique qui décrit l'arbre de filiation
    while len(species) > 1:
        # on cherche le couple d'espèces les plus proches
        closer1, closer2 = minimal_couple(distances, species)
        # on peut les enlever de notre radar
        species.remove(closer1)
        species.remove(closer2)
        # et en faire une espèce synthétique
        new_species = closer1, closer2
        # il faut recalculer les distances entre cette nouvelle
        # espèce et les espèces encore en lice
        # cf le transparent :
        # dist(F,C),A = (dist F,A + dist C,A) / 2 
        for sp in species:
            distances[ (sp, new_species) ] = \
              (get_distance(distances, closer1, sp) + \
               get_distance(distances, closer2, sp)) / 2
        # on peut maintenant ajouter la nouvelle espèce
        # pour le tour suivant
        species.append(new_species)
        # bavardage
        if verbose:
            print(10*'=', "species = ", species)
            print(distances)
    # le résultat est le seul élément de species
    return species[0]

### Sur un exemple simple

Avec les données de `data/named-species.txt`, à nouveau&nbsp;:

In [ ]:
with open("data/named-species.txt") as input:
    for line in input:
        print(line, end="")

On obtient ce résultat&nbsp;:

In [ ]:
UPGMA("data/named-species.txt")

Ou encore, en version bavarde&nbsp;:

In [ ]:
UPGMA("data/named-species.txt", True)

### Un exemple un peu plus réaliste

Voyons à présent le résultat obteun avec des données un peu plus réalistes:

In [ ]:
with open("data/upgma-input.txt") as input:
    for line in input:
        print(line, end="")

In [ ]:
UPGMA("data/upgma-input.txt")